# MLlib

Spark biedt ook een framework aan voor MachineLearning modellen te trainen op gedistribueerde datasets.
Dit framework is MLlib of ook wel sparkML genoemd.
De code om te werken met deze package is sterk gelijkaardig aan sklearn.
De API en een uitgebreide documentatie met voorbeeldcode kan je [hier](https://spark.apache.org/docs/latest/ml-guide.html) vinden.

Deze package bied de volgende tools aan
* ML-technieken: classificatie, regressie, clustering, ...
* Features: Extracting en transforming van features, PCA, ...
* Pipelines: Maak, train, optimaliseer en evalueer pipelines
* Persistentie: Bewaar en laden van algoritmes/modellen
* Databeheer: Algebra tools, statistieken, null-waarden, ...

Let op dat er twee API's aangeboden worden, 1 gebaseerd op RDD's en 1 op DataFrames.
De API gebaseerd op RDD's is ouder en minder flexibel dan de API gebruik makend van DataFrames.
Momenteel werken ze allebei maar in de toekomst zou de RDD gebaseerde kunnen verdwijnen.

## Utilities

### Varianten voor numpy-arrays

Voor feature sets en volledige matrices van datasets aan te maken kan je gebruik maken van de Vector en Matrix klassen.
Deze beschikken over een Dense variant waar je elk element moet ingeven of een Sparse Variant waar cellen, elementen leeg kan laten.
Dit ziet er als volgt uit:

In [6]:
from pyspark.sql import SparkSession
from pyspark.ml.linalg import Vectors, Matrices

spark = SparkSession.builder.appName("MLlib les").getOrCreate()

# 1ste index heeft de waarde 20 en 3de index heeft de waarde 40
data  = Vectors.sparse(4, [(1, 20.0), (3, 40)])
print(data)
data = Vectors.dense([10.0, 20.0, 30.0, 40.0])
print(data)

# 4 rijen, 5 kolommen
data = Matrices.dense(4, 5, range(20))
print(data)

(4,[1,3],[20.0,40.0])
[10.0,20.0,30.0,40.0]
DenseMatrix([[ 0.,  4.,  8., 12., 16.],
             [ 1.,  5.,  9., 13., 17.],
             [ 2.,  6., 10., 14., 18.],
             [ 3.,  7., 11., 15., 19.]])


Het is belangrijk om te weten dat dit locale datastructuren (wrapper rond numpy array) zijn en geen gedistribueerde objecten.

### Statistieken

Voor er kan gewerkt worden met statistieken moeten we (net zoals bij pandas) eerst een dataset hebben.
Hieronder maken we een random dataframe aan van 50 rijen en 4 kolommen.

In [11]:
from pyspark.mllib.random import RandomRDDs

# 50 rijen, 4 kolommen. we moeten eerst ook alle waardes naar een list veranderen om die te kunnen omzetten in een DF.
data = RandomRDDs.uniformVectorRDD(spark.sparkContext, 50, 4).map(lambda a: a.tolist()).toDF()
#data.show()
data.describe().show()
data.summary().show()

+-------+-------------------+--------------------+--------------------+-------------------+
|summary|                 _1|                  _2|                  _3|                 _4|
+-------+-------------------+--------------------+--------------------+-------------------+
|  count|                 50|                  50|                  50|                 50|
|   mean|0.45562355825191936|  0.4645917480810971|  0.5332524347932288|0.42927580322421505|
| stddev| 0.2850113494113795|  0.2945111017051597| 0.29985438726453856|0.28703798803216446|
|    min|0.01248083893147145|0.026382482425437503|0.022861220918282688|0.00383515376189536|
|    max| 0.9347140171756639|  0.9997115724800131|  0.9985109965285944| 0.9415570066595731|
+-------+-------------------+--------------------+--------------------+-------------------+



+-------+-------------------+--------------------+--------------------+-------------------+
|summary|                 _1|                  _2|                  _3|                 _4|
+-------+-------------------+--------------------+--------------------+-------------------+
|  count|                 50|                  50|                  50|                 50|
|   mean|0.45562355825191936|  0.4645917480810971|  0.5332524347932288|0.42927580322421505|
| stddev| 0.2850113494113795|  0.2945111017051597| 0.29985438726453856|0.28703798803216446|
|    min|0.01248083893147145|0.026382482425437503|0.022861220918282688|0.00383515376189536|
|    25%|0.19421395124502927| 0.19483220200892803|  0.2783252273304532|0.19577999724053952|
|    50%| 0.4195192842572516|  0.4122525307526367|  0.5625772445702768| 0.3968339578536181|
|    75%| 0.7062213780991908|  0.7137135703132439|  0.7647202734224299| 0.6941042848608009|
|    max| 0.9347140171756639|  0.9997115724800131|  0.9985109965285944| 0.941557

**Correlation matrix**

Buiten de statistieken die berekend kunnen worden door de summary() functie kan ook de correlatiematrix belangrijk zijn.
Deze matrix maakt het mogelijk om het verband tussen de verscheidene features te bestuderen.
Deze matrix kan als volgt berekend worden voor een gedistribueerd dataframe.

In [17]:
# eerst alles omzetten naar vector
from pyspark.ml.feature import VectorAssembler
# een assembler zal alle kolommen merge in een vector
assembler = VectorAssembler(inputCols = data.columns, outputCol = "vector")
data_vector = assembler.transform(data)
#data.show(5)
#data_vector.show(5)

from pyspark.ml.stat import Correlation

df_corr = Correlation.corr(data_vector, "vector")
df_corr.show(truncate=False)

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|pearson(vector)                                                                                                                                                                                                                                                                                                                                                       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

**Onafhankelijksheidtest**

Naast de correlatiematrix kan het ook belangrijk zijn om de onafhankelijkheid te testen tussen elke feature en een label.
Dit kan uitgevoerd worden door een zogenaamde ChiSquareTest.
Deze krijgt als input een dataframe, de naam van de kolom met de features (als vectors) en de naam van een kolom met de labels.
We kunnen deze test uitvoeren als volgt:

In [22]:
from pyspark.sql.functions import rand, when

# voeg een label toe op basis van een nieuw willekeurig getal
# we gaan kijken of het label afhankelijk is van de features in de vector kolom
data_label = data_vector.withColumn("label", when(rand() > .5, 1).otherwise(0)).show()

from pyspark.ml.stat import ChiSquareTest
data_chi2test = ChiSquareTest.test(data_label, "vector", "label")
data_chi2test.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+-----+
|                  _1|                  _2|                  _3|                  _4|              vector|label|
+--------------------+--------------------+--------------------+--------------------+--------------------+-----+
|  0.6786402569272866|  0.1875351488551662|  0.7141094941050921| 0.23480925096652705|[0.67864025692728...|    1|
|  0.3286454952102197|  0.5940405697895388|  0.5124647148012205|  0.6076959751870075|[0.32864549521021...|    0|
|  0.8037966893694491|  0.3062193135304848|  0.6966550546898116|  0.9415570066595731|[0.80379668936944...|    0|
|  0.3650753306057878| 0.19483220200892803| 0.08238258562457268| 0.21781337015488966|[0.36507533060578...|    1|
|0.021164107524203057|  0.7498019921828105|  0.6946474816967066| 0.39718601540300325|[0.02116410752420...|    1|
|  0.3469408072004708|  0.3060211128026944|  0.2784172925138798| 0.20652801111047625|[0.34694080

Py4JJavaError: An error occurred while calling z:org.apache.spark.ml.stat.ChiSquareTest.test.
: java.lang.NullPointerException
	at org.apache.spark.ml.stat.ChiSquareTest$.test(ChiSquareTest.scala:75)
	at org.apache.spark.ml.stat.ChiSquareTest.test(ChiSquareTest.scala)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at java.base/jdk.internal.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.base/java.lang.reflect.Method.invoke(Method.java:566)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
	at py4j.Gateway.invoke(Gateway.java:282)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.ClientServerConnection.waitForCommands(ClientServerConne

**Summarizer**

Andere statistieken per kolom kunnen berekend worden door gebruik te maken van de Summarizer klasse:

In [38]:
from pyspark.ml.stat import Summarizer

summarizer = Summarizer.metrics("mean", "count", "max")

df_convert = data_vector.select(summarizer.summary(data_vector.vector)).show(truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|aggregate_metrics(vector, 1.0)                                                                                                                                       |
+---------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|{[0.45562355825191936,0.46459174808109704,0.5332524347932288,0.42927580322421505], 50, [0.9347140171756639,0.9997115724800131,0.9985109965285944,0.9415570066595731]}|
+---------------------------------------------------------------------------------------------------------------------------------------------------------------------+



Het gebruik maken van de Summarizer maakt het dus mogelijk om rechtstreeks op de feature vectors te werken zonder ze eerst terug te moeten splitsen.

### Pipelines

Pipelines binnen Spark zijn een groep van high-level API's steunend op Dataframes om ML-pipelines aan te maken, optimaliseren en trainen.
De belangrijkste concepten binnen de Pipelines van Spark zijn:
* Dataframe: concept van de dataset
* Transformer: Zet een dataframe om in een ander dataframe
* Estimator: Zet een dataframe om in een model/transformer
* Pipeline: een ketting van transformers en estimators om een flow vast te leggen
* Parameter: API voor parameters van transformers en estimators aan te passen

Gebruik nu onderstaande mini-dataset waar we op basis van een tekstkolom met logistische regressie een bepaald label proberen te voorspellen.
Maak hiervoor een Pipeline uit die bestaat uit de volgende stappen:
* Tokenizer om de tekstkolom te splitsen in de overeenkomstige woorden
* HashingTf om de term frequency van de woorden te bepalen en het om te zetten naar een feature vector
* LogisticRegression Estimator om de voorspelling te doen.

Train daarna deze pipeline en maak de voorspellingen voor de traningsdata.
Hoe accuraat is dit model?

In [50]:
# Prepare training documents from a list of (id, text, label) tuples.
# classificatie: zit "spark" in de tekst -> 1 is true, 0 is false
training = spark.createDataFrame([
    (0, "a b c d e spark", 1.0),
    (1, "b d", 0.0),
    (2, "spark f g h", 1.0),
    (3, "hadoop mapreduce", 0.0)
], ["id", "text", "label"])

from pyspark.ml import Pipeline
from pyspark.ml.feature import Tokenizer, HashingTF
from pyspark.ml.classification import LogisticRegression
# stap 1 -> splits tekst apart in lists (standaard door spatie)
tokenizer = Tokenizer(inputCol="text", outputCol="words")

hasher = HashingTF(inputCol="words", outputCol="features")
# resultaat = (262144,[74920,89530,107107,148981,167694,173558],[1.0,1.0,1.0,1.0,1.0,1.0])|
# dit is een sparse vector
# 262144 aantal elementen/mogelijke hashes
# woord met hash 7490 staat er 1 keer in, 89530 ook 1 keer in, ...
# dit is eigenlijk een bag of words
lr = LogisticRegression()

pipeline = Pipeline(stages=[tokenizer, hasher, lr])
#hasher.transform(tokenizer.transform(training)).show(truncate=False)
model = pipeline.fit(training)
predictions = model.transform(training)
predictions.show(truncate=False)



+---+----------------+-----+----------------------+----------------------------------------------------------------------------+----------------------------------------+-----------------------------------------+----------+
|id |text            |label|words                 |features                                                                    |rawPrediction                           |probability                              |prediction|
+---+----------------+-----+----------------------+----------------------------------------------------------------------------+----------------------------------------+-----------------------------------------+----------+
|0  |a b c d e spark |1.0  |[a, b, c, d, e, spark]|(262144,[74920,89530,107107,148981,167694,173558],[1.0,1.0,1.0,1.0,1.0,1.0])|[-19.141960614724145,19.141960614724145]|[4.861296721081615E-9,0.9999999951387033]|1.0       |
|1  |b d             |0.0  |[b, d]                |(262144,[89530,148981],[1.0,1.0])                        

### Evalueren van een model

In de pyspark.ml package zitten er ook functionaliteiten voor deze modellen te evalueren.
Meer informatie hierover vind je [hier](https://spark.apache.org/docs/2.2.0/mllib-evaluation-metrics.html).

In [51]:
# evalueren van het model
from pyspark.ml.evaluation import BinaryClassificationEvaluator

evaluator = BinaryClassificationEvaluator()
evaluator.evaluate(predictions)
# output: 1.0 -> 100% juist

1.0

### Data sources

Door gebruik te maken van de sparkContext kunnen een reeks standaard databronnen ingelezen worden om datasets uit op te bouwen (Csv, Json, ...).
Daarnaast is het ook mogelijk om een folder met een reeks beelden te gebruiken als dataset om zo een model voor image classification te trainen.
Download nu [deze](https://www.kaggle.com/returnofsputnik/chihuahua-or-muffin) dataset en upload ze naar een folder op het hadoop filesysteem.

In [52]:
# download dataset
import opendatasets as od
od.download("https://www.kaggle.com/returnofsputnik/chihuahua-or-muffin")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: segmentation
Your Kaggle Key: ········


100%|████████████████████████████████████████| 183k/183k [00:00<00:00, 3.35MB/s]

In [54]:
# upload dataset
import pydoop.hdfs as hdfs
localFS = hdfs.hdfs(host='')
client=hdfs.hdfs(host="localhost", port=9000)

if not client.exists("/user/bigdata/06_Spark"):
    client.create_directory('/user/bigdata/06_Spark')
client.set_working_directory('/user/bigdata/06_Spark')

localFS.copy("chihuahua-or-muffin", client, "chihuahua-or-muffin")

0

De geuploade images kunnen nu ingelezen worden als volgt:

In [58]:
# read images
df = spark.read.format("image").option("dropInvalid", True).load("06_Spark/chihuahua-or-muffin")
df.show(2)
df.printSchema()
df.select("image.origin", "image.height", "image.width").show(truncate=False)

+--------------------+
|               image|
+--------------------+
|{hdfs://localhost...|
|{hdfs://localhost...|
+--------------------+
only showing top 2 rows

root
 |-- image: struct (nullable = true)
 |    |-- origin: string (nullable = true)
 |    |-- height: integer (nullable = true)
 |    |-- width: integer (nullable = true)
 |    |-- nChannels: integer (nullable = true)
 |    |-- mode: integer (nullable = true)
 |    |-- data: binary (nullable = true)

+-------------------------------------------------------------------------------+------+-----+
|origin                                                                         |height|width|
+-------------------------------------------------------------------------------+------+-----+
|hdfs://localhost:9000/user/bigdata/06_Spark/chihuahua-or-muffin/muffin-4.jpeg  |170   |172  |
|hdfs://localhost:9000/user/bigdata/06_Spark/chihuahua-or-muffin/muffin-7.jpeg  |172   |171  |
|hdfs://localhost:9000/user/bigdata/06_Spark/chihuahua-or-m

Merk op dat het werken met images niet zo eenvoudig is.
Hiervoor wordt binnen pyspark typisch gebruik gemaakt van de [sparkdl](https://smurching.github.io/spark-deep-learning/site/api/python/sparkdl.html) package.
Hierbij staat de dl voor deep learning.
Aangezien dit ons momenteel te ver leidt ga ik dit niet verder toelichten.

Een andere aparte databron die eenvoudig ingelezen kan worden is het formaat "libsvm".
Een bestand van dit formaat wordt ingelezen als een dataframe met twee kolommen: een label en een kolom met de feature-vectors.
De code om dergelijk bestand in te laden is:

In [ ]:
df = spark.read.format("libsvm").load("{path to file here}")